In [1]:
import cobra

# import the BiGG core model 
bigg_model_path = '..\COBRA function scripts\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)

# Parse the supplementary thermodynamic dataset

In [2]:
'''import pandas
import re

# acquire Gibbs reactions data from the TMFA supplementary excel file        
reactions_data = pandas.read_excel('Supplementary file.xls', sheet_name = 'Reactions')
reactions_data.columns = reactions_data.iloc[0]
reactions_data.drop(reactions_data.index[0], axis = 0, inplace = True)
reactions_data.head()
reactions_dict = {}
for index, row in reactions_data.iterrows():
    reaction_abbreviation = reactions_data.at[index, 'iJR904 Abbreviation']
    reaction_name = reactions_data.at[index, 'Reaction name']
    reaction_gibbs = reactions_data.at[index, 'Estimated Gibbs free energy change of reaction (kcal/mol)']

    reactions_dict[reaction_abbreviation] = {'name': reaction_name, 'gibbs': reaction_gibbs}

# acquire Gibbs compounds data from the TMFA supplementary excel file        
compounds_data = pandas.read_excel('Supplementary file.xls', sheet_name = 'Compounds')
compounds_dict = {}
for index, row in compounds_data.iterrows():
    compound_abbreviation = compounds_data.at[index, 'iJR904 Abbreviation']
    compound_name = compounds_data.at[index, 'Compound Name']
    compound_charge = compounds_data.at[index, 'Charge at pH 7*']
    try:
        compound_gibbs = float(compounds_data.at[index, 'Estimated Gibbs free energy of formation (kcal/mol)']) / calorie
    except:
        compound_gibbs = 0

    compounds_dict[compound_abbreviation] = {'name': compound_name, 'gibbs': compound_gibbs, 'charge': compound_charge}'''

"import pandas\nimport re\n\n# acquire Gibbs reactions data from the TMFA supplementary excel file        \nreactions_data = pandas.read_excel('Supplementary file.xls', sheet_name = 'Reactions')\nreactions_data.columns = reactions_data.iloc[0]\nreactions_data.drop(reactions_data.index[0], axis = 0, inplace = True)\nreactions_data.head()\nreactions_dict = {}\nfor index, row in reactions_data.iterrows():\n    reaction_abbreviation = reactions_data.at[index, 'iJR904 Abbreviation']\n    reaction_name = reactions_data.at[index, 'Reaction name']\n    reaction_gibbs = reactions_data.at[index, 'Estimated Gibbs free energy change of reaction (kcal/mol)']\n\n    reactions_dict[reaction_abbreviation] = {'name': reaction_name, 'gibbs': reaction_gibbs}\n\n# acquire Gibbs compounds data from the TMFA supplementary excel file        \ncompounds_data = pandas.read_excel('Supplementary file.xls', sheet_name = 'Compounds')\ncompounds_dict = {}\nfor index, row in compounds_data.iterrows():\n    compound_

# Aerobic media

In [3]:
# display the model media
message = 'model media'
print(message, '\n', '='*len(message))
display(model.medium)

# define the model solver
model.solver = 'cplex'

# display the model results
model.summary()

model media 


{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


# Anaerobic media

In [4]:
# edit the model media
message = 'model media'
print(message, '\n', '='*len(message))
media = model.medium
media['EX_o2_e'] = 0
model.medium = media
display(model.medium)

# define the model solver
model.solver = 'cplex'

# display the model results
model.summary()

model media 


{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,0.3782,1,0.63%
glc__D_e,EX_glc__D_e,10,6,99.37%
h2o_e,EX_h2o_e,7.116,0,0.00%
nh4_e,EX_nh4_e,1.154,0,0.00%
pi_e,EX_pi_e,0.7786,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-8.504,2,33.11%
etoh_e,EX_etoh_e,-8.279,2,32.23%
for_e,EX_for_e,-17.8,1,34.66%
h_e,EX_h_e,-30.55,0,0.00%


# FVA 

In [5]:
from cobra.flux_analysis import flux_variability_analysis

flux_variability_analysis(model, model.reactions)

,minimum,maximum
PFK,9.789459,9.789459
PFL,17.804674,17.804674
PGI,9.956609,9.956609
PGK,-19.437336,-19.437336
PGL,0.000000,0.000000
...,...,...
NADH16,0.000000,0.000000
NADTRHD,0.000000,0.000000
NH4t,1.154156,1.154156
O2t,0.000000,0.000000


# Imposing thermodynamic constraints

In [6]:
# import general libraries
import pandas

# import the multiTFA
from multitfa.core import tmodel

#display(dir(cobra.io))

# importing the core E.coli model 
def build_core_model():
    from cobra.io import read_sbml_model
    
    model = read_sbml_model("e_coli_core.xml")
    pH_I_T_dict = {
        "pH": {"c": 7.5, "e": 7, "p": 7},
        "I": {"c": 0.25, "e": 0, "p": 0},
        "T": {"c": 298.15, "e": 298.15, "p": 298.15},
    }
    del_psi_dict = {
        "c": {"c": 0, "e": 0, "p": 150},
        "e": {"c": 0, "e": 0, "p": 0},
        "p": {"c": -150, "e": 0, "p": 0},
    }
    del_psi = pandas.DataFrame.from_dict(data=del_psi_dict)
    comp_info = pandas.DataFrame.from_dict(data=pH_I_T_dict)

    Excl = [rxn.id for rxn in model.boundary] + [
        "BIOMASS_Ecoli_core_w_GAM",
        "O2t",
        "H2Ot",
    ]

    tfa_model = tmodel(
        model, Exclude_list=Excl, compartment_info=comp_info, membrane_potential=del_psi
    )
    for met in tfa_model.metabolites:
        kegg_id = "bigg.metabolite:" + met.id[:-2]
        met.Kegg_id = kegg_id

    tfa_model.update()

    return tfa_model


# exporting a TSV file of the model flux ranges
def main():
    from multitfa.analysis import variability_legacy_cplex
    from cobra import Configuration
    
    config = Configuration()
    config.solver = "cplex"
    tfa_model = build_core_model()
    vars_analysis = [
        rxn.id for rxn in tfa_model.reactions if not rxn.id.startswith("DM_")
    ]
    ranges = variability_legacy_cplex(tfa_model, variable_list=vars_analysis)

    ranges.to_csv("ranges_ecoli_cplex.tsv", header=True, index=True, sep="\t")
    
    
# creating the multiTFA model instance
multitfa_model = build_core_model()
main()

# exporting the LP file
with open('multiTFA_LP_file.lp', 'w') as out:
    out.write(str(multitfa_model.solver))

Fragments already downloaded
Fragments already downloaded
Fragments already downloaded

Selected objective sense:  MINIMIZE
Selected objective  name:  d19075d5-df9f-11eb-af72-50eb71d9aedf
Selected RHS        name:  rhs
Selected bound      name:  bnd

Selected objective sense:  MINIMIZE
Selected objective  name:  d19075d5-df9f-11eb-af72-50eb71d9aedf
Selected RHS        name:  rhs
Selected bound      name:  bnd


# Aerobic media

In [7]:
# display the model media
message = 'model media'
print(message, '\n', '='*len(message))
display(multitfa_model.medium)

# define the model solver
multitfa_model.solver = 'cplex'

# display the model results
multitfa_model.summary()

model media 


{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


# Anaerobic media

In [8]:
# edit the model media
message = 'model media'
print(message, '\n', '='*len(message))
media = multitfa_model.medium
media['EX_o2_e'] = 0
multitfa_model.medium = media
display(multitfa_model.medium)

# define the model solver
multitfa_model.solver = 'cplex'

# display the model results
multitfa_model.summary()

model media 


{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,0.3782,1,0.63%
glc__D_e,EX_glc__D_e,10,6,99.37%
h2o_e,EX_h2o_e,7.116,0,0.00%
nh4_e,EX_nh4_e,1.154,0,0.00%
pi_e,EX_pi_e,0.7786,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-8.504,2,33.11%
etoh_e,EX_etoh_e,-8.279,2,32.23%
for_e,EX_for_e,-17.8,1,34.66%
h_e,EX_h_e,-30.55,0,0.00%


# FVA 

The below code has consistently crashed my code. This occurs with any thermodynamically-constrained model, including the Full-Thermo Package. 

In [ ]:
'''from cobra.flux_analysis import flux_variability_analysis

flux_variability_analysis(multitfa_model, multitfa_model.reactions)'''